In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time

In [2]:
PAUSE_TIME = 2
driver = webdriver.Chrome(r'C:\Users\Chun\chromedriver_win32\chromedriver.exe')

#     create initial urls for each clothing categories
category_list = ['activewear','dresses','jeans','lingerie-sleepwear','tops','workwear-1']
for category in category_list:
    link = f'https://www.saksoff5th.com/c/women/apparel/{category}'
    driver.get(link)